In [1]:
%load_ext autoreload
%autoreload 2

from PyQt6.QtWidgets import QFileDialog, QErrorMessage
from layout import ApplicationLayout
import pandas, numpy
from core.data import mapping, extract
from itertools import product
from collections import Counter
from core.optimizer import optimize
from core.data.schema import WeightMinMax, DataFrame
from core.data import transform

flat_effects = [eff for eff in mapping.runes.effects.values() if not eff.endswith('%')]
data = pandas.DataFrame({"weight": 1, "min": pandas.NA, "max": pandas.NA}, dtype="Int64", index=flat_effects)
empty = "Empty"

class Application(ApplicationLayout):
    def __init__(self):
        super().__init__(minw=1200)
        self.json = None
        self.monsters = None
        self.runes = None

        self.weight_min_max.data = data.T
        self.monster_selector.addItem("Monster")
        self.monster_selector.setDisabled(True)
        for sel in self.runesets_selectors:
            sel.addItems([empty, *sorted(mapping.runes.sets.values())])

        self.button.clicked.connect(self.select_data_file)
        # self.enable_optimizer()


    def gatherSets(self):
        return dict(Counter([sel.value for sel in self.runesets_selectors if sel.value != empty]))

    def gatherWeightMinMax(self) -> DataFrame[WeightMinMax]:
        wmm = self.weight_min_max.data.astype("Int64").T
        wmm.weight = wmm.weight.fillna(0)
        return wmm

    def select_data_file(self, *args):
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.FileMode.ExistingFile)
        file_dialog.setNameFilter("JSON Files (*.json)")
        file_dialog.setViewMode(QFileDialog.ViewMode.List)
        if file_dialog.exec():
            file_paths = file_dialog.selectedFiles()
            if file_paths:
                file_path = file_paths[0]  # Get the first selected file
                self.json = extract.json(file_path)
                self.enable_optimizer()

    def enable_optimizer(self):
        self.runes = extract.runes(self.json)
        self.monsters = extract.monsters(self.json)
        self.monster_selector.setEnabled(True)
        self.monster_selector.removeItem(0)
        self.monster_selector.addItems(sorted(self.monsters.name.dropna().values))
        self.button.setText("Optimize")
        self.button.clicked.disconnect(self.select_data_file)
        self.button.clicked.connect(self.optimize)

    def optimize(self):
        selected_monsters = transform.get_monsters_by_name(self.monsters, self.monster_selector.value)
        filtered_runes = self.runes 
        if not self.allow_equipped_checkbox.isChecked():
            filtered_runes = transform.filter_equipped(self.runes, selected_monsters)
        solution = optimize(filtered_runes, selected_monsters.iloc[:1], self.gatherWeightMinMax(), self.gatherSets())
        if solution is not None:
            self.results.data = transform.named_monster_runes_view(solution, self.monsters).set_index("slot").drop(columns="spd%")
        else:
            QErrorMessage(self).showMessage("The formulated program is infeasible.")

app = Application()
app.exec()

               value effect  stacked
Energy          1680     hp     True
Fatal            215    atk    False
Blade             12    crt     True
Swift             24    spd    False
Focus             20    acc     True
Guard            117    def     True
Endure            20    res     True
Rage              40    crd    False
Fight             49    atk     True
Determination     62    def     True
Enhance          896     hp     True
Accuracy          10    acc     True
Tolerance         10    res     True
hp 1680*Energy_0 + 1680*Energy_1 + 1680*Energy_2 + 896*Enhance_0 + 896*Enhance_1 + 896*Enhance_2
atk 215*Fatal_0 + 49*Fight_0 + 49*Fight_1 + 49*Fight_2
crt 12*Blade_0 + 12*Blade_1 + 12*Blade_2
spd 24*Swift_0
acc 10*Accuracy_0 + 10*Accuracy_1 + 10*Accuracy_2 + 20*Focus_0 + 20*Focus_1 + 20*Focus_2
def 62*Determination_0 + 62*Determination_1 + 62*Determination_2 + 117*Guard_0 + 117*Guard_1 + 117*Guard_2
res 20*Endure_0 + 20*Endure_1 + 20*Endure_2 + 10*Tolerance_0 + 10*Tolerance_1 